# 10.1 텍스트 임베딩 이해하기

- 문장을 벡터로 변환하는 방식

## 10.1.1 문장 임베딩 방식의 장점

- 데이터의 의미를 숫자로 표현할 수 있다면, 데이터가 서로 유사한지, 관련이 있는지와 같이 중요한 정보를 활용할 수 있다.
- 이번 예제에서는 문장 임베딩 방식을 사용하면 단어나 문장 사이의 관계를 계산할 수 있다는 점만 이해해도 충분하다.
- 원핫 인코딩, 백오브워즈, TF-IDF, 워드투벡을 순서대로 알아본다.

In [ ]:
# 예제 10.1 문장 임베딩을 활용한 단어 간 유사도 계산

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

smodel = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')  # https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
smodel # 일반적으로 work_embedding_dimension은 주로 768, 1024 차원으로 설정된다. (차원은 의미를 담을 수 있는 공간), 256의 배수로 표현되는 것 같은데 이유는 모르겠음

In [ ]:
dense_embeddings = smodel.encode(['학교', '공부', '운동'])
cosine_similarity(dense_embeddings)

## 10.1.2 원핫 인코딩

- 학교, 공부, 운동을 예시로 아래와 같이 표현하는 것을 원핫 인코딩이라고 한다.

| 단어 |  |  |  |
|---|--|---|---|
| 학교 | 1 | 0 | 0 |
| 공부 | 0 | 1 | 0 |
| 운동 | 0 | 0 | 1 |

- 이 방식을 사용하면 '식사'라는 새로운 데이터를 추가해도 아래처럼 독립적으로 추가할 수 있고, 단어와 단어 사이에 아무런 관계도 나타내지 않는다.

| 단어 |  |  |  |  |
|---|--|---|---|---|
| 학교 | 1 | 0 | 0 | 0 |
| 공부 | 0 | 1 | 0 | 0 |
| 운동 | 0 | 0 | 1 | 0 |
| 식사 | 0 | 0 | 0 | 1 |

### 장점
- 범주형 데이터 사이에 의도하지 않은 관계가 담기는 걸 방지한다는 장점

### 단점
- 충분히 관련이 있는 단어 사이의 관계도 표현할 수 없다는 단점

In [ ]:
# 예제 10.2 원핫 인코딩의 한계

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

word_dict = {
    "school": np.array([[1, 0, 0]]),
    "study": np.array([[0, 1, 0]]),
    "workout": np.array([[0, 1, 0]]),
}
cosine_similarity(word_dict['school'], word_dict['study'])

In [ ]:
cosine_similarity(word_dict['school'], word_dict['workout'])

## 10.1.2 백오브워즈

- 백오브워즈(Bag Of Words)는 '비슷한 단어가 많이 나오면 비슷한 문장 또는 문서'라는 가정을 활용해 문서를 숫자로 변환한다.

| | 가계 대출 | 증시 | AI | 부동산 | LLM | 구글 |
|---|-------|----|----|-----|-----|----|
| 경제 기사 1 | 3     | 3  | 0  | 2   | 0   | 0  |
| 경제 기사 2 | 0     | 5  | 3  | 1   | 0   | 0  |
| IT 기사 1 | 0     | 0  | 3  | 0   | 4   | 2  |
| IT 기사 2 | 0     | 0  | 2  | 1   | 2   | 0  |

### 장점
- 아이디어가 직관적이고 구현이 간단함에도 훌륭히 작동하기 때문에 문장과 문서의 의미를 표현하는 방법으로 오랫동안 사용
### 단점
- 어떤 단어가 많이 나왔다고 해서 문서의 의미를 파악하는데 크게 도움이 되지 않는 경우가 있다는 단점
    - 예를 들어, 조사('은/는/이/가', '을/를')는 거의 모든 한국어 문서에 등장 (불용어 전처리 필요)
- `AI`라는 단어는 여러 기사에서 언급하기 때문에 `AI`라는 단어가 등장했다는 사실만으로는 문서의 의미를 예측하기 어려움

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['i want to go home', 'i want to go work']
vector = CountVectorizer()

print('BOW Vector', vector.fit_transform(corpus).toarray())
print('BOW Vocabulary', vector.vocabulary_)

## 10.1.3 TF-IDF
- TF-IDF(Term Frequency-Inverse Document Frequency)는 백오브워즈의 단점을 보완하기 위해 등장한 방법
$$ \mathrm {TFIDF} (w)=\mathrm{TF}(w) \times \log(N / \mathrm{DF}(w)) $$
- TF(w): 문서에서 단어 w의 빈도
- DF(w): 단어 w가 등장한 문서의 수
- N: 전체 문서의 수

| | TF("이") | TF("LLM") | DF("이") | DF("LLM") | TF-IDF("이") | TF-IDF("LLM") |
|---|---------|-----------|---------|-----------|-------------|--------------|
| 경제 기사 1 | 10      | 0       | 4       | 2         | 0           | 0            |
| 경제 기사 2 | 8       | 0       | 4       | 2         | 0           | 0            |
| IT 기사 1 | 5       | 4       | 4       | 2         | 0           | 4 * log(4/2) |
| IT 기사 2 | 9       | 2         | 4       | 2         | 0           | 2 * log(4/2) |

- 조사 '이'는 모든 문서에 등장하기 때문에 TF-IDF("이")는 0이 된다. (중요도가 없다.)
- 백오브워즈의 문제를 성공적으로 보완하면서 오랫동안 활발히 사용
    - 이후에 설명하는 BM25(Best Matching 25)와 같은 TF-IDF의 변형 방식이 현재까지도 가장 보편적인 연관도 점수 계산 방식으로 사용


## 10.1.4 워드투벡
- 워드투벡(Word2Vec)은 단어가 `함께 등장하는 빈도` 정보를 활용해 단어의 의미를 압축하는 단어 임베딩 방법
  - "AI"는 "ML" 또는 "머신러닝"
  - "한강"은 "라면"이나 "자전거"

---

- 특정 단어 주변에 어떤 단어가 있는지 예측하는 모델을 만든다면 단어의 의미를 표현한 임베딩을 모델(인공신경망)이 생성할 수 있지 않을까 하는 가정
  - 주변 단어로 중간 단어를 예측하는 방식(CBOW, Continuous Bagof Words)
  - 중간 단어로 주변 단어를 예측하는 방식(Skip-Gram)

![image.png](../resources/Word2Vec-architecture.png)
![image.png](../resources/Word2Vec-data.png)

---

- CBOW는 주변의 단어 정보로 중간에 있을 단어를 예측하는 방식
    - t번째 단어를 예측하기 위해 위아래로 2개의 단어 정보를 활용 (t-2, t-1, t+1, t+2)

![image.png](../resources/Word2Vec-cbow.png)

---

- Skip-Gram은 중간 단어로 주변 단어를 예측하는 방식
    - t번째 단어를 중심으로 위아래로 2개의 단어를 예측 (t-2, t-1, t+1, t+2)

![image.png](../resources/Word2Vec-skipgram.png)
![image.png](../resources/Word2Vec-skipgram2.png)

---

- 벡터간 유사도를 계산하여 단어 사이의 관계/의미를 확인할 수 있다.

![image.png](../resources/Word2Vec-relation.jpeg)